In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import timedelta
import time

In [3]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_SLOW_PERIOD = 26
MACD_FAST_PERIOD = 12
MACD_SIGNAL_PERIOD = 9
initial_investment = 100_000

In [4]:
non_ESG = ["GME", "FOSL", "GT", "HOG", "PDCO"]

In [5]:
high_ESG =  ['CCU','CNH','TIETO.HE','BAYN.BE','SGSNZ.XC']

In [6]:
medium_esg = ['VOC','VRSK','19MA.F','QNBK.QA','1605.TW']

In [7]:
all_comp = high_ESG + medium_esg + non_ESG

In [8]:
def calculate_indicators_macd(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['Close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [9]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [10]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [11]:
def simulate_investment_macd(ticker):
    try:
        time.sleep(12)
        data = yf.Ticker(ticker).history(start='2014-01-01', end='2016-01-01')
        data = calculate_indicators_macd(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+1 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            if current_date in buy_signals:
                # Invest 
                allocation = cash 
                if allocation > 0:
                    shares_to_buy = int(allocation // data['Close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['Close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            shares_to_buy = int(cash // data['Close'][i])
                            total_cost = shares_to_buy * data['Close'][i]
                            cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(1), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

            # Check if we need to sell due to sell signal
            current_price = data['Close'].iloc[i]
            if holdings > 0:
                should_sell = False

                # Check sell signal
                if current_date in sell_signals:
                    should_sell = True

                if should_sell:
                    shares_to_sell = int(holdings)
                    if shares_to_sell > 0:
                        if shares_to_sell > holdings:
                            shares_to_sell = holdings
                        revenue = shares_to_sell * current_price 
                        holdings -= shares_to_sell
                        settlement_date = get_next_trading_day(current_date + BDay(1), trading_days)
                        if settlement_date in pending_sell_revenue:
                            pending_sell_revenue[settlement_date] += revenue
                        else:
                            pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['Close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(1):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['Close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [12]:
def backtest_macd(all_comp):
    results = []
    for company in all_comp:
        result = simulate_investment_macd(company)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment)  * 100
            })
    return pd.DataFrame(results)

In [13]:
macd_res = backtest_macd(all_comp)

C:\Users\Admin\AppData\Local\Temp\ipykernel_36496\3326753981.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_36496\3326753981.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [14]:
macd_res

,Company,Final Portfolio Value,Total Profit,Rate of Return
0,CCU,80133.211133,-19866.788867,-19.866789
1,CNH,52961.894661,-47038.105339,-47.038105
2,TIETO.HE,109180.749034,9180.749034,9.180749
3,BAYN.BE,142394.515892,42394.515892,42.394516
4,SGSNZ.XC,109240.974747,9240.974747,9.240975
5,VOC,52718.345528,-47281.654472,-47.281654
6,VRSK,112489.472828,12489.472828,12.489473
7,19MA.F,193867.584953,93867.584953,93.867585
8,QNBK.QA,114176.023527,14176.023527,14.176024
9,1605.TW,74844.570094,-25155.429906,-25.155430


In [15]:
macd_res.to_excel('decimalodd_res/macd.xlsx')

In [16]:
def calculate_indicators_bb(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['Close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['Close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [17]:
def bb_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    df.loc[
        (df['Close'] <= df['Bollinger_low']) &
        (df['RSI'] <= RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] >= RSI_OVERBOUGHT) &
        (df['Close'] >= df['Bollinger_high']), 'Signal'] = -1

    return df

In [18]:
def simulate_investment_bb(ticker):
    try:
        time.sleep(12)
        data = yf.Ticker(ticker).history(start='2014-01-01', end='2016-01-01')
        # data = data.set_index(pd.DatetimeIndex(data['Date'].values))
        data = calculate_indicators_bb(data)
        data = bb_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+1 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            if current_date in buy_signals:
                # Invest 
                allocation = cash 
                if allocation > 0:
                    shares_to_buy = int(allocation // data['Close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['Close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            shares_to_buy = int(cash // data['Close'][i])
                            total_cost = shares_to_buy * data['Close'][i]
                            cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(1), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

            # Check if we need to sell due to sell signal
            current_price = data['Close'].iloc[i]
            if holdings > 0:
                should_sell = False

                # Check sell signal
                if current_date in sell_signals:
                    should_sell = True

                if should_sell:
                    shares_to_sell = int(holdings)
                    if shares_to_sell > 0:
                        if shares_to_sell > holdings:
                            shares_to_sell = holdings
                        revenue = shares_to_sell * current_price 
                        holdings -= shares_to_sell
                        settlement_date = get_next_trading_day(current_date + BDay(1), trading_days)
                        if settlement_date in pending_sell_revenue:
                            pending_sell_revenue[settlement_date] += revenue
                        else:
                            pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['Close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(1):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['Close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [19]:
def backtest_bb(all_comp):
    results = []
    for company in all_comp:
        result = simulate_investment_bb(company)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment)  * 100
            })
    return pd.DataFrame(results)

In [20]:
bb_res = backtest_bb(all_comp)

C:\Users\Admin\AppData\Local\Temp\ipykernel_36496\999902845.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_36496\1496493288.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = int(allocation // data['Close'][i])
C:\Users\Admin\AppData\

In [21]:
bb_res.to_excel('decimalodd_res/bb.xlsx')

In [22]:
OBV_PERIOD = 5

In [23]:
def calculate_indicators_obv(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

def strategy_obv(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI across 30 and OBV rise
    df.loc[(df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0), 'Signal'] = 1

    #Sell Signals: RSI across 70 and OBV down 
    df.loc[(df['Previous_RSI'] > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) & (df['OBV_Slope'] < 0), 'Signal'] = -1

    return df


In [24]:
def simulate_investment_obv(ticker):
    try:
        time.sleep(12)
        data = yf.Ticker(ticker).history(start='2014-01-01', end='2016-01-01')
        # data = data.set_index(pd.DatetimeIndex(data['Date'].values))
        data = calculate_indicators_obv(data)
        data = strategy_obv(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        portfolio_values = []

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i in range(len(data)):
            current_date = data.index[i]

            # Update pending transactions (T+1 settlement)
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            if current_date in buy_signals:
                # Invest 
                allocation = cash 
                if allocation > 0:
                    shares_to_buy = int(allocation // data['Close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['Close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            shares_to_buy = int(cash // data['Close'][i])
                            total_cost = shares_to_buy * data['Close'][i]
                            cash -= total_cost

                    settlement_date = get_next_trading_day(current_date + BDay(1), trading_days)
                    if settlement_date in pending_buy_shares:
                        pending_buy_shares[settlement_date] += shares_to_buy
                    else:
                        pending_buy_shares[settlement_date] = shares_to_buy

            # Check if we need to sell due to sell signal
            current_price = data['Close'].iloc[i]
            if holdings > 0:
                should_sell = False

                # Check sell signal
                if current_date in sell_signals:
                    should_sell = True

                if should_sell:
                    shares_to_sell = int(holdings)
                    if shares_to_sell > 0:
                        if shares_to_sell > holdings:
                            shares_to_sell = holdings
                        revenue = shares_to_sell * current_price 
                        holdings -= shares_to_sell
                        settlement_date = get_next_trading_day(current_date + BDay(1), trading_days)
                        if settlement_date in pending_sell_revenue:
                            pending_sell_revenue[settlement_date] += revenue
                        else:
                            pending_sell_revenue[settlement_date] = revenue

            current_value = cash + holdings * data['Close'].iloc[i]
            portfolio_values.append(current_value)

        # Calculate final portfolio value including pending transactions
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(1):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)

            current_value = cash + holdings * data['Close'].iloc[-1]
            portfolio_values.append(current_value)
            final_date += BDay(1)

        # Ensure the length of portfolio_values matches the length of data index
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [25]:
def backtest_obv(all_comp):
    results = []
    for company in all_comp:
        result = simulate_investment_obv(company)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment)  * 100
            })
    return pd.DataFrame(results)

In [26]:
obv_res = backtest_obv(all_comp)
obv_res.to_excel('decimalodd_res/obv.xlsx')

C:\Users\Admin\AppData\Local\Temp\ipykernel_36496\2826085594.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_36496\4076441267.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = int(allocation // data['Close'][i])
C:\Users\Admin\AppData